<a href="https://colab.research.google.com/github/hossainsiam133/BanglaTaka/blob/main/Code/Siam/BanglaTaka_Siam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mount Google Drive**

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import os
import shutil
import random
output_dir = "/content/drive/MyDrive/dataset_Split"


**Load Dataset**

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224, 224)
batch_size = 32

train_gen = ImageDataGenerator(rescale=1./255)
val_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    os.path.join(output_dir, "train"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_data = val_gen.flow_from_directory(
    os.path.join(output_dir, "val"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_data = test_gen.flow_from_directory(
    os.path.join(output_dir, "test"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 6300 images belonging to 9 classes.
Found 1350 images belonging to 9 classes.
Found 1350 images belonging to 9 classes.


**Build the Model (VGG19)**

In [5]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# Load pretrained VGG19 without top layers
base_model = VGG19(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze pretrained layers
base_model.trainable = False

# Build model
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(9, activation='softmax')  # 9 classes
])

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg19 (Functional)              │ (None, 7, 7, 512)      │    20,024,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 9)              │         2,313 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,449,481 (100.90 MB)

 Trainable params: 6,425,097 (24.51 MB)

 Non-trainable params: 20,024,384 (76.39 MB)

**Train the Model**

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=15
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
  8/197 ━━━━━━━━━━━━━━━━━━━━ 1:11:25 23s/step - accuracy: 0.3530 - loss: 1.9173

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

**Save the Model**

In [ ]:
model.save("/content/drive/MyDrive/BanglaTaka_VGG19.h5")
print("✅ Model saved to Drive")

✅ Model saved to Drive


Let's evaluate the model on the test dataset to get a better understanding of its overall performance.

**Evaluate on Test Set**

In [ ]:
loss, accuracy = model.evaluate(test_data)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


43/43 ━━━━━━━━━━━━━━━━━━━━ 104s 2s/step - accuracy: 0.9918 - loss: 0.0405
Test Loss: 0.0462
Test Accuracy: 0.9911


Please upload the image you would like to classify using the widget below.

**Test On a Single Image**

In [ ]:
from google.colab import files
from io import BytesIO
from PIL import Image as PILImage
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import os

uploaded = files.upload()

for fn in uploaded.keys():

    img_bytes = uploaded[fn]
    img_stream = BytesIO(img_bytes)
    img = PILImage.open(img_stream)


    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0


    if 'model' not in globals():
        model_path = "/content/drive/MyDrive/dataset_model_final.h5"
        model = tf.keras.models.load_model(model_path)
        print("\n✅ Model loaded for prediction.")


    if 'train_data' in globals():
        class_labels = list(train_data.class_indices.keys())
    else:

        print("Warning: train_data not found. Attempting to infer class labels from output_dir.")
        output_dir = '/content/drive/MyDrive/dataset_Split'
        train_dir = os.path.join(output_dir, "train")
        class_labels = sorted(os.listdir(train_dir))


    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class_label = class_labels[predicted_class_index]
    confidence = predictions[0][predicted_class_index]

    print(f"\nUploaded image '{fn}':")
    print(f"Predicted class: {predicted_class_label}")
    print(f"Confidence: {confidence:.2f}")



Saving AM.jpg.jpeg to AM.jpg (1).jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step

Uploaded image 'AM.jpg (1).jpeg':
Predicted class: 100
Confidence: 0.98
